# CAICT 風機結冰預測 - (DNNAutoencoder)

2017/07/21   
徐仕杰

## Outline

-  [Load Data and Module](#load) 
-  [1. Import MNIST Data](#Import Data) 
-  [2. seMMA- Autoencoder](#開始Deep Learning)  
-  [3. Reference](#reference)

<a id='load'></a>
## Load Data and Module

In [1]:
import pandas as pd
import os
import sys
from PIL import Image
import numpy as np
import lzma
import pickle
from IPython.display import display
import tensorflow as tf
from tfdot import tfdot
from tensorflow.contrib.tensorboard.plugins import projector
import shutil
import matplotlib.pyplot as plt
import shutil
import csv
import sklearn as sk
from sklearn.metrics import roc_curve, auc 
from sklearn.metrics import precision_score
import matplotlib.gridspec as gridspec
import seaborn as sns
from pylab import *

In [3]:
def updateProgress(msg):
    sys.stdout.write('\r')
    sys.stdout.write(msg)
    sys.stdout.flush()

In [4]:
def variable_summaries(var, name):  
    with tf.name_scope('summaries_'+str(name)):  
        mean = tf.reduce_mean(var)  
        tf.summary.scalar('mean', mean)  
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))  
        tf.summary.scalar('stddev', stddev)  
        tf.summary.scalar('max', tf.reduce_max(var))  
        tf.summary.scalar('min', tf.reduce_min(var))  
        tf.summary.histogram('histogram', var)  

<a id='Import MNIST Data'></a>
## 1. Import MNIST Data

#### 先把MNIST資料讀進來
- Training Data: 訓練Model
- Validataion Data: 訓練Model的時候, 同步監控目前模型的好壞
- Testing Data: 訓練結束後, 評估模型的好壞

In [5]:
with lzma.open("mnist.pkl.xz", 'rb') as f:
    train_set, validation_set, test_set = pickle.load(f, encoding='latin1')

print('list裡的前面是picture X',train_set[0])
print('後面是label Y',train_set[1])


list裡的前面是picture X [[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
後面是label Y [5 0 4 ..., 8 4 8]


In [6]:
train_X, train_y = train_set
validation_X, validation_y = validation_set
test_X, test_y = test_set
print('training data size:',len(train_X))
print('validataion data size:',len(validation_X))
print('testing data size:',len(test_X))
print('picture shape:',train_X[0].shape)

training data size: 50000
validataion data size: 10000
testing data size: 10000
picture shape: (784,)


#### 把Y label變成one-hot representation

In [7]:
train_Y = np.eye(10)[train_y]
test_Y = np.eye(10)[test_y]
validation_Y = np.eye(10)[validation_y]

<a id='開始Deep Learning'></a>
## 2. Autoencoder

** DNN, Deep Neural Network (就是很多層的f(WX+B)) **


** What is autoencoder ? **

** autoencoder可以幹嘛？ **


- Deep Learning ABC 
    -  [A. 定義參數](#定義參數) 
    -  [B. 設計一個Model從X預測Y](#設計一個) 
    -  [C. 選一個loss function,](#選一個loss) 
    -  [D. 選一個optimizer](#選一個o) 
    -  [E. 開始執行訓練](#開始執行) 
    -  [F. 算一下正確率](#算一下正)  

<a id='定義參數'></a>
### A. 定義參數(Placeholder, Variable, Constant)
tips: 把要餵進Model的資料X,Y定義成placeholder, 把要讓電腦幫忙找的權重W,B定義成Variable

In [8]:
# hyperparameters
n_inputs = 784 # 每一行的维度
n_classes = 10  # RNN最后的输出類別個数

** autoencoder **

In [9]:
encoded_neural = 20
tf.reset_default_graph()
X =tf.placeholder(tf.float32, [None, n_inputs], name="X")
Y_ = tf.placeholder(tf.float32, shape=[None, 10], name="Y_")

W = {
    'enc_wd1': tf.Variable(tf.random_normal([784,300], stddev=0.01), name="enc_wd1"),
    'enc_wd2': tf.Variable(tf.random_normal([300,100], stddev=0.01), name="enc_wd2"),
    'enc_wd3': tf.Variable(tf.random_normal([100,encoded_neural], stddev=0.01), name="enc_wd3"),
    'dec_wd1': tf.Variable(tf.random_normal([encoded_neural,100], stddev=0.01), name="dec_wd1"),
    'dec_wd2': tf.Variable(tf.random_normal([100,300], stddev=0.01), name="dec_wd2"),
    'dec_wd3': tf.Variable(tf.random_normal([300,784], stddev=0.01), name="dec_wd3"),
}

B = {
    'enc_bd1': tf.Variable(tf.random_normal([300]),name="enc_bd1"),
    'enc_bd2': tf.Variable(tf.random_normal([100]), name="enc_bd2"),
    'enc_bd3': tf.Variable(tf.random_normal([encoded_neural]), name="enc_bd2"),
    'dec_bd1': tf.Variable(tf.random_normal([100]), name="dec_bd1"),
    'dec_bd2': tf.Variable(tf.random_normal([300]),name="dec_bd2"),
    'dec_bd3': tf.Variable(tf.random_normal([784]), name="dec_bd3"),
}


** classification **

In [10]:
W2 = {
    'cla_wd1': tf.Variable(tf.random_normal([encoded_neural,10], stddev=0.01), name="cla_wd1"),
}
B2 = {
    'cla_bd1': tf.Variable(tf.random_normal([10]),name="cla_bd1"),
}
trainable_vars = [W2['cla_wd1'],B2['cla_bd1']]

<a id='設計一個'></a>
###  B. 設計一個Model從X預測X^ 
Input Layer: X  
Hidden Layer 1: $H_1=f_1(W_1X+B_1)$  
Hidden Layer 2: $H_2=f_2(W_2H_1+B_2)$  
Output Layer : $Y=f_3(W_3H_2+B_3)$  


### L1. Input Layer 輸入層
- do nothing

### L2. Hidden Layer 隱藏層 x 2
- 線性(WX+B) + 非線性(activation function)  


In [11]:
with tf.name_scope("Hidden_Layer_enc1"):
    H1 = tf.matmul(X, W['enc_wd1']) + B['enc_bd1'] 
    H1 = tf.nn.sigmoid(H1, name="H1")

In [12]:
with tf.name_scope("Hidden_Layer_enc1"):
    H2 = tf.matmul(H1, W['enc_wd2']) + B['enc_bd2'] 
    H2 = tf.nn.sigmoid(H2, name="H2")

In [13]:
with tf.name_scope("Hidden_Layer_enc3"):
    encoded = tf.matmul(H2, W['enc_wd3']) + B['enc_bd3'] 
    encoded = tf.nn.sigmoid(encoded, name="encoded")

### L3. Output Layer 輸出層

In [14]:
with tf.name_scope("Hidden_Layer_dec1"):
    H10 = tf.matmul(encoded, W['dec_wd1']) + B['dec_bd1'] 
    H10 = tf.nn.relu(H10, name="H10")

In [15]:
B['dec_bd2'] 

<tf.Variable 'dec_bd2:0' shape=(300,) dtype=float32_ref>

In [16]:
with tf.name_scope("Hidden_Layer_dec2"):
    H11 = tf.matmul(H10, W['dec_wd2']) + B['dec_bd2'] 
    H11 = tf.nn.relu(H11, name="H11")

In [17]:
with tf.name_scope("Hidden_Layer_dec2"):
    decoded = tf.matmul(H11, W['dec_wd3']) + B['dec_bd3'] 
    decoded = tf.nn.sigmoid(decoded, name="decoded")

### L4 Classification Layer 

In [18]:
Y= tf.add(x= tf.matmul(encoded, W2['cla_wd1'], name='e'), y=B2['cla_bd1'], name='Y')
Y_softmax = tf.nn.softmax(Y, name='Y_softmax') #這個是one-hot的格式
Y_softmax_argmax = tf.argmax(Y_softmax, axis=1) #從one-hot的格式找出最大那個
correct_prediction = tf.equal(Y_softmax_argmax, tf.argmax(Y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))


<a id='選一個loss'></a>
###  C. Define loss function & Optimizer
- cross_entorpy $-log(\Pr(Y_{true}))$

** Autoencoder **

In [19]:
# Cost Function basic term
cross_entropy = -1. * X * tf.log(decoded) - (1. - X) * tf.log(1. - decoded)
loss = tf.reduce_mean(cross_entropy)
optimizer = tf.train.RMSPropOptimizer(0.01).minimize(loss)

** Classification **

In [20]:
loss_cla = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y_, logits=Y))
optimizer_cla = tf.train.RMSPropOptimizer(0.01).minimize(loss_cla,var_list=trainable_vars)

<a id='開始執行'></a>
### E1. Training Autoencoder

In [21]:
init = tf.global_variables_initializer()
sess = tf.InteractiveSession()
sess.run(init)

In [22]:
epoch = 20
batch_size = 100
total_batch= len(train_X) / batch_size
for ep in range(epoch+1):
    for i in range(int(total_batch)+1):
        rnd_idx = np.random.choice(train_X.shape[0], batch_size, replace=False)
        batch_x = train_X[rnd_idx]
        #batch_y = train_Y[rnd_idx]
        _, loss_v= sess.run([optimizer, loss], feed_dict={X: batch_x})
        if i%100 ==0:
            #loss_s, acc_s, summary= sess.run([loss], feed_dict={X: validation_X })
            updateProgress('epoch:{x0}, batch:{x4} loss:{x3}'.format(x0=ep,x3=loss_v,x4=i))
    print()


epoch:0, batch:500 loss:0.24213919043540955
epoch:1, batch:500 loss:0.20510199666023254
epoch:2, batch:500 loss:0.18776308000087738
epoch:3, batch:500 loss:0.17192156612873077
epoch:4, batch:500 loss:0.16977192461490633
epoch:5, batch:500 loss:0.16077344119548798
epoch:6, batch:500 loss:0.15106005966663364
epoch:7, batch:500 loss:0.15095590054988866
epoch:8, batch:500 loss:0.14694295823574066
epoch:9, batch:500 loss:0.15180905163288116
epoch:10, batch:500 loss:0.14934760332107544
epoch:11, batch:500 loss:0.14046455919742584
epoch:12, batch:500 loss:0.13366372883319855
epoch:13, batch:500 loss:0.12876971065998077
epoch:14, batch:500 loss:0.13402418792247772
epoch:15, batch:500 loss:0.13103510439395905
epoch:16, batch:500 loss:0.13083803653717044
epoch:17, batch:500 loss:0.12092138081789017
epoch:18, batch:500 loss:0.12661527097225198
epoch:19, batch:500 loss:0.12462119758129122
epoch:20, batch:500 loss:0.12173298746347427


In [23]:
showX(test_X[:20])
batch_tx = test_X[:20]
decoded_imgs = sess.run([decoded], feed_dict={X: batch_tx})

decoded_imgs_1 = np.array(decoded_imgs)
decoded_imgs_1 = decoded_imgs_1.reshape(20,784)
print('')
showX(decoded_imgs_1[:20])

### other picture

In [24]:
?im.resize

Object `im.resize` not found.


In [36]:
from PIL import Image
piclist = ['img/ae1.jpg','img/ae2.jpg','img/ae3.jpg','img/ae4.jpeg','img/ae5.jpeg']
batch_tx=[]
for pic in piclist:
    im = Image.open(pic).convert('L') #Can be many different formats.
    #display(im)
    im = im.resize((28,28))
    display(im)
    pix = im.load()
    pixels = list(im.getdata())
    pixels = np.array(pixels).reshape(784,)
    batch_tx.append(pixels)
decoded_img_test = sess.run([decoded], feed_dict={X: batch_tx})
decoded_img_test = np.array(decoded_img_test)
decoded_img_test = decoded_img_test.reshape(-1,784)
showX(decoded_img_test[:20])

### E2. Training Classification

In [26]:
epoch = 15
batch_size = 128
total_batch= len(train_X) / batch_size
for ep in range(epoch+1):
    for i in range(int(total_batch)+1):
        rnd_idx = np.random.choice(train_X.shape[0], batch_size, replace=False)
        batch_x = train_X[rnd_idx]
        batch_y = train_Y[rnd_idx]
        _, loss_v, acc_v= sess.run([optimizer_cla, loss_cla, accuracy], feed_dict={X: batch_x, Y_:batch_y})
        if i%100 ==0:
            loss_v2, acc_v2= sess.run([loss_cla, accuracy], feed_dict={X: validation_X , Y_: validation_Y})
            updateProgress('Ep:{x0}, Bch:{x4}, Tr: loss:{x5}, acc:{x6}, Vd: loss:{x3}, acc:{x2}'.format(x0=ep,x1=i,x2=acc_v2,x3=loss_v2,x4=i,x6=acc_v,x5=loss_v))
    print()



Ep:0, Bch:300, Tr: loss:0.987533688545227, acc:0.8046875, Vd: loss:0.871279776096344, acc:0.845700025558471748
Ep:1, Bch:300, Tr: loss:0.6405439376831055, acc:0.8359375, Vd: loss:0.46304750442504883, acc:0.8899999856948853
Ep:2, Bch:300, Tr: loss:0.3318834900856018, acc:0.875, Vd: loss:0.37347957491874695, acc:0.905600011348724460083
Ep:3, Bch:300, Tr: loss:0.38083508610725403, acc:0.90625, Vd: loss:0.33773213624954224, acc:0.909300029277801547
Ep:4, Bch:300, Tr: loss:0.26936668157577515, acc:0.9375, Vd: loss:0.31922224164009094, acc:0.9136999845504761008
Ep:5, Bch:300, Tr: loss:0.3781154155731201, acc:0.921875, Vd: loss:0.30730709433555603, acc:0.9143000245094299
Ep:6, Bch:300, Tr: loss:0.22317060828208923, acc:0.9453125, Vd: loss:0.2999330461025238, acc:0.9162999987602234
Ep:7, Bch:300, Tr: loss:0.3465901017189026, acc:0.90625, Vd: loss:0.2955923080444336, acc:0.91729998588562017285
Ep:8, Bch:300, Tr: loss:0.3159705400466919, acc:0.9375, Vd: loss:0.2890099585056305, acc:0.91809999942

### Accuracy

In [27]:
# 抓出前五筆訓練資料的預測結果 Y_softmax (注意：這個是one-hot的格式)
pred_tmp = Y_softmax.eval(feed_dict={X: train_X[21:30]})
pred_tmp_df = pd.DataFrame(pred_tmp)
_ = pred_tmp_df.apply(np.argmax,axis=1)
print('predict: ',_.values)
showX(train_X[21:30])
print('answer: ',train_y[21:30])
_ = correct_prediction.eval({X: train_X[21:30] , Y_: train_Y[21:30]})
print('Training 第21~30筆資料的比對結果:',pd.DataFrame(_))

predict:  [0 9 1 1 2 4 3 2 7]


answer:  [0 9 1 1 2 4 3 2 7]
Training 第21~30筆資料的比對結果:       0
0  True
1  True
2  True
3  True
4  True
5  True
6  True
7  True
8  True


In [38]:
# 抓出前五筆訓練資料的預測結果 Y_softmax (注意：這個是one-hot的格式)
pred_tmp = Y_softmax.eval(feed_dict={X: test_X[:100]})
pred_tmp_df = pd.DataFrame(pred_tmp)
_ = pred_tmp_df.apply(np.argmax,axis=1)
print('predict: ',_.values)
showX(test_X[:100])
batch_tx = test_X[:100]
decoded_imgs = sess.run([decoded], feed_dict={X: batch_tx})
decoded_imgs_1 = np.array(decoded_imgs)
decoded_imgs_1 = decoded_imgs_1.reshape(-1,784)
showX(decoded_imgs_1)
print('answer: ',train_y[:100])
_ = correct_prediction.eval({X: test_X[:100] , Y_: test_Y[:100]})
print('Testing 第21~30筆資料的比對結果:',pd.DataFrame(_))

predict:  [6 4 5 4 0 7 4 0 1]


answer:  [5 0 4 1 9 2 1 3 1 4 3 5 3 6 1 7 2 8 6 9 4 0 9 1 1 2 4 3 2 7 3 8 6 9 0 5 6
 0 7 6 1 8 7 9 3 9 8 5 9 3 3 0 7 4 9 8 0 9 4 1 4 4 6 0 4 5 6 1 0 0 1 7 1 6
 3 0 2 1 1 7 9 0 2 6 7 8 3 9 0 4 6 7 4 6 8 0 7 8 3 1]
Testing 第21~30筆資料的比對結果:         0
0    True
1    True
2    True
3    True
4    True
5    True
6    True
7    True
8    True
9    True
10   True
11   True
12   True
13   True
14   True
15   True
16   True
17   True
18   True
19   True
20   True
21   True
22  False
23   True
24   True
25   True
26   True
27   True
28   True
29   True
..    ...
70   True
71   True
72   True
73  False
74   True
75   True
76   True
77  False
78   True
79   True
80  False
81   True
82   True
83   True
84   True
85   True
86   True
87   True
88   True
89   True
90   True
91   True
92   True
93   True
94   True
95   True
96   True
97   True
98  False
99   True

[100 rows x 1 columns]


In [29]:
acc = sess.run(accuracy, feed_dict={X: test_X , Y_: test_Y})
print('Accuracy:',acc)

Accuracy: 0.9149


### Tensor Borad

In [30]:
LOG_DIR = 'tb-log0'
input_data = train_X[:5000]
input_y = train_y[:5000]
if not os.path.exists(LOG_DIR):
    os.mkdir(LOG_DIR)
#把圖片的label存到metadata.tsv
metadata_file_path = os.path.join(LOG_DIR, 'metadata.tsv')
with open(metadata_file_path, 'w') as metadata_file:
    for row in range(len(input_y)):
        c=train_y[row]
        metadata_file.write('{}\n'.format(c))

In [31]:
#Save variable
Input_Var = tf.Variable(input_data, name='Input_var')
tmp_aevalue = encoded.eval({X: input_data})
encoded_Var = tf.Variable(encoded.eval({X: input_data}), name='encoded_var')
decoded_Var = tf.Variable(decoded.eval({X: input_data}), name='decoded_Var')

saver = tf.train.Saver([Input_Var,encoded_Var,decoded_Var])
sess.run(Input_Var.initializer)
sess.run(encoded_Var.initializer)
sess.run(decoded_Var.initializer)
saver.save(sess, os.path.join(LOG_DIR, 'autoencoder_train.ckpt'))

#在config裡面用一個embedding關聯 tensor & its metadata
config = projector.ProjectorConfig()
# One can add multiple embeddings.

embedding1 = config.embeddings.add()
embedding1.tensor_name = Input_Var.name
embedding1.metadata_path = metadata_file_path

embedding2 = config.embeddings.add()
embedding2.tensor_name = encoded_Var.name
embedding2.metadata_path = metadata_file_path

embedding3 = config.embeddings.add()
embedding3.tensor_name = decoded_Var.name
embedding3.metadata_path = metadata_file_path

#embedding4 = config.embeddings.add()
#embedding4.tensor_name = Out_Var.name
#embedding4.metadata_path = metadata_file_path

#embedding.sprite.image_path = os.path.join(LOG_DIR, 'img/mnist_10k_sprite.png')
# Specify the width and height of a single thumbnail.
#embedding.sprite.single_image_dim.extend([28, 28])



# Saves a config file that TensorBoard will read during startup.
projector.visualize_embeddings(tf.summary.FileWriter(LOG_DIR), config)


In [32]:
#!tensorboard --logdir=tb-log0 --port=6006